# Explore the National Water Model Reanalysis v2.1 


### CASE 1: each HDF file contains chunks in multiple variables; doesn't need h5py

370k files.

In [1]:
%%time
import fsspec
import xarray as xr
mapper = fsspec.get_mapper(
    "reference://",
    fo="s3://esip-qhub-public/noaa/nwm/nwm_reanalysis.json.zst",
    target_options={
        "anon": True,
        "compression": "zstd"
    },
    remote_options={"anon": True}
)
ds = xr.open_dataset(mapper, engine="zarr", backend_kwargs={"consolidated": False})

/Users/mdurant/conda/envs/py38/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'qBtmVertRunoff' has multiple fill values {-9999000, 0}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/Users/mdurant/conda/envs/py38/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'qBucket' has multiple fill values {-999900000, 0}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/Users/mdurant/conda/envs/py38/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'qSfcLatRunoff' has multiple fill values {-999900000, 0}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/Users/mdurant/conda/envs/py38/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'q_lateral' has multiple fill values {0, -99990}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/Users/mdurant/conda/envs/py38/lib/python3.8/site-packages/

CPU times: user 38.5 s, sys: 1.56 s, total: 40.1 s
Wall time: 45.3 s


In [2]:
ds

<xarray.Dataset>
Dimensions:         (time: 367439, feature_id: 2776738)
Coordinates:
  * feature_id      (feature_id) float64 101.0 179.0 181.0 ... 1.18e+09 1.18e+09
    latitude        (feature_id) float32 ...
    longitude       (feature_id) float32 ...
  * time            (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T...
Data variables:
    elevation       (time, feature_id) float32 ...
    order           (time, feature_id) float64 ...
    qBtmVertRunoff  (time, feature_id) float64 ...
    qBucket         (time, feature_id) float64 ...
    qSfcLatRunoff   (time, feature_id) float64 ...
    q_lateral       (time, feature_id) float64 ...
    streamflow      (time, feature_id) float64 ...
    velocity        (time, feature_id) float64 ...
Attributes: (12/19)
    Conventions:                CF-1.6
    TITLE:                      OUTPUT FROM WRF-Hydro v5.2.0-beta2
    _NCProperties:              version=2,netcdf=4.7.4,hdf5=1.10.7,
    cdm_datatype:               Station
    code_version:               v5.2.0-beta2
    dev:                        dev_ prefix indicates development/internal me...
    ...                         ...
    model_output_type:          channel_rt
    model_output_valid_time:    1979-02-01_01:00:00
    model_total_valid_times:    1416
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    station_dimension:          feature_id
    stream_order_output:        1

In [3]:
ds.nbytes/1e12, ds.streamflow.nbytes/1e12 # How many terabytes total/in a single variable

(61.21695740624, 8.162254671856)

#### Read and plot streamflow for a specific time 
The local National Weather Service office in Houston observed all-time record daily rainfall accumulations on both August 26 and 27, measured at 14.4 in (370 mm) and 16.08 in (408 mm) respectively

In [ ]:
import hvplot.pandas
import geoviews as gv
from holoviews.operation.datashader import rasterize
import cartopy.crs as ccrs
import numpy as np
import pandas as pd

In [ ]:
ds1 = ds.sel(time='2017-08-27 18:00:00', method='nearest')

In [ ]:
var = 'streamflow'

In [ ]:
df = ds1[var].to_pandas().to_frame()

In [ ]:
date_title = pd.to_datetime(ds1.time.values).strftime('%Y-%m-%d %H-%M-%D')
date_title = f'{var}: {date_title}'
date_title

In [ ]:
df = df.assign(latitude=ds.latitude)
df = df.assign(longitude=ds.longitude)
df.rename(columns={0: var}, inplace=True)

In [ ]:
p = df.hvplot.points(x='longitude', y='latitude', geo=True,
                     c=var, colorbar=True, size=14, label=date_title)
g = rasterize(p, aggregator='mean', x_sampling=0.02, 
                y_sampling=0.02, width=500).opts(tools=['hover'], 
                aspect='equal', logz=True, cmap='viridis', clim=(1e-2, np.nan))
g * gv.tile_sources.OSM